## Look at different connectomes from different stages from Wilvliet and others. 

In [ ]:
## Plot various sta?ges of the connectome from Wilvliet

In [ ]:
from cedne import utils
import matplotlib.pyplot as plt
import numpy as np

## Loading adult

In [ ]:
worm = {}
for j in range(1,3):
    import_parameters = {'style':'witvliet', 'stage':'adult', 'dataset_ind':str(j)}
    worm[j] = utils.makeWorm(name = f"{import_parameters['style']}_{import_parameters['stage']}_{import_parameters['dataset_ind']}", import_parameters=import_parameters)
    n = worm[j].networks[f"{import_parameters['style']}_{import_parameters['stage']}_{import_parameters['dataset_ind']}"]
    print(len(n.connections))

In [ ]:
networks_to_join = []
for j in range(2,3):
    networks_to_join.append(worm[j].networks[f"{import_parameters['style']}_{import_parameters['stage']}_{j}"])

n1 = worm[1].networks[f"{import_parameters['style']}_{import_parameters['stage']}_{1}"]
joined_adult = n1.join_networks(networks_to_join)
joined_adult.remove_unconnected_neurons()

In [ ]:
# import_parameters = {'style':'witvliet', 'stage':'adult', 'dataset_ind':'1'}
# w = utils.makeWorm(import_parameters=import_parameters)
# nn_adult = w.networks["witvliet_adult_1"]

## Loading L1

In [ ]:
worm = {}
for j in range(1,5):
    import_parameters = {'style':'witvliet', 'stage':'L1', 'dataset_ind':str(j)}
    worm[j] = utils.makeWorm(name = f"{import_parameters['style']}_{import_parameters['stage']}_{import_parameters['dataset_ind']}", import_parameters=import_parameters)

In [ ]:
networks_to_join = []
for j in range(2,5):
    networks_to_join.append(worm[j].networks[f"{import_parameters['style']}_{import_parameters['stage']}_{j}"])

n1 = worm[1].networks[f"{import_parameters['style']}_{import_parameters['stage']}_{1}"]
joined_l1 = n1.join_networks(networks_to_join)
joined_l1.remove_unconnected_neurons()

## Loading L2

In [ ]:
import_parameters = {'style':'witvliet', 'stage':'L2', 'dataset_ind':'1'}
w = utils.makeWorm(import_parameters=import_parameters)
nn_l2 = w.networks["witvliet_L2_1"]

## Loading L3

In [ ]:
import_parameters = {'style':'witvliet', 'stage':'L3', 'dataset_ind':'1'}
w = utils.makeWorm(import_parameters=import_parameters)
nn_l3 = w.networks["witvliet_L3_1"]

## Finding neurons that connect to HSN neurons.

In [ ]:
conns_adult = []
for e in joined_adult.neurons['HSNL'].get_connections(direction='out').keys():
    conns_adult.append(e[1].name)
for e in joined_adult.neurons['HSNR'].get_connections(direction='out').keys():
    conns_adult.append(e[1].name)
for e in joined_adult.neurons['HSNL'].get_connections(direction='in').keys():
    conns_adult.append(e[0].name)
for e in joined_adult.neurons['HSNR'].get_connections(direction='in').keys():
    conns_adult.append(e[0].name)
conns_adult = set(conns_adult)

In [ ]:
present_subset = set(joined_l1.neurons) & set(nn_l2.neurons) & set(nn_l3.neurons) & set(joined_adult.neurons)

## HSN connected network
l1_subnet = joined_l1.subnetwork([n for n in conns_adult if n in present_subset])
l2_subnet = nn_l2.subnetwork([n for n in conns_adult if n in  present_subset])
l3_subnet = nn_l3.subnetwork([n for n in conns_adult if n in  present_subset])
adult_subnet = joined_adult.subnetwork([n for n in conns_adult if n in present_subset])

## Rest of the network
l1_residual = joined_l1.subnetwork([n for n in present_subset if not n in conns_adult])
l2_residual = nn_l2.subnetwork([n for n in present_subset if not n in conns_adult])
l3_residual = nn_l3.subnetwork([n for n in present_subset if not n in conns_adult])
adult_residual = joined_adult.subnetwork([n for n in present_subset if not n in conns_adult])

In [ ]:
connectivity_subnet = {}
for net in [l1_subnet, l2_subnet, l3_subnet, adult_subnet]:
    for n in sorted(net.neurons):
        if not n in connectivity_subnet:
            connectivity_subnet[n] = []
        connectivity_subnet[n].append(len(net.neurons[n].get_connections(direction='both')))

connectivity_residual = {}
for net in [l1_residual, l2_residual, l3_residual, adult_residual]:
    for n in sorted(net.neurons):
        if not n in connectivity_residual:
            connectivity_residual[n] = []
        connectivity_residual[n].append(len(net.neurons[n].get_connections(direction='both')))

In [ ]:
f, ax = plt.subplots(figsize=(4,2), ncols=2, sharey=True, sharex=True, layout='constrained')
for n in connectivity_subnet:
    ax[0].plot(connectivity_subnet[n], color='gray')
ax[0].set_xticks([0,1,2,3], ['L1', 'L2', 'L3', 'Adult'])

for n in connectivity_residual:
    ax[1].plot(connectivity_residual[n], color='gray')
ax[1].set_xticks([0,1,2,3], ['L1', 'L2', 'L3', 'Adult'])
f.supylabel("# Connections per neuron")

utils.simpleaxis(ax)
ax[0].set_title("HSN connected")
ax[1].set_title("Other neurons")
plt.show()

## Looking at triad motifs

In [ ]:
triad_motifs = utils.return_triads()
motif = triad_motifs['030T']
motif = utils.nx.relabel_nodes(motif, {1:1, 2:3, 3:2})

In [ ]:
ntype = ['sensory', 'interneuron', 'motorneuron']
facecolors = ['#FF6F61', '#FFD700', '#4682B4']
ntype_pairs = set([tuple(sorted([nt1, nt2])) for nt1 in ntype for nt2 in ntype])
colors= plt.cm.magma(np.linspace(0,1,len(ntype_pairs)))
type_color_dict = {p:color for (p,color) in zip(ntype_pairs, colors)}

In [ ]:
## All common neurons present.

l1_all = joined_l1.subnetwork(present_subset)
l2_all = nn_l2.subnetwork(present_subset)
l3_all = nn_l3.subnetwork(present_subset)
adult_all = joined_adult.subnetwork(present_subset)

# network_dict = {stage:net for stage, net in zip(['L1', 'L2', 'L3', 'Adult'], [l1_all, l2_all, l3_all, adult_all])}

In [ ]:
network_dict = {stage:net for stage, net in zip(['L1', 'L2', 'L3', 'Adult'], [joined_l1, nn_l2, nn_l3, joined_adult])}

In [ ]:
adult_default = utils.makeWorm()
nn = adult_default.networks["Neutral"]

In [ ]:
for stage, net in network_dict.items():
    for n in net.neurons:
        if n == 'DB1':
            m = 'DB01'
        else:
            m=n
        net.neurons[n].type = nn.neurons[m].type

In [ ]:
mappings = [(2,1), (3,1)]
num_graphs = {}
conn_types = {stage:{mapping:{n:0 for n in ntype_pairs} for mapping in mappings} for stage in network_dict.keys()}
ntype_motif = {}
max_chain_length = 6
for stage, net in network_dict.items():
    num_graphs[stage] = {}
    ntype_motif[stage] = {}
    for mapping in mappings:
        num_graphs[stage][mapping] = []
        ntype_motif[stage][mapping] = {}
        for l in range(1,max_chain_length+1):
            hm = utils.make_hypermotifs(motif, l, [mapping])
            for m in hm.nodes:
                ntype_motif[stage][mapping][m] = {n:0 for n in ntype}
            all_ffgs = net.search_motifs(hm)
            num_graphs[stage][mapping].append(len(all_ffgs))
            for sub in all_ffgs:
                for motif_edge, network_edge in sub.items():
                    if all(n.type in ntype for n in network_edge):
                        for m,n in zip(motif_edge, network_edge):
                            ntype_motif[stage][mapping][m][n.type] +=1
                        conn = tuple(sorted([network_edge[0].type, network_edge[1].type]))
                        conn_types[stage][mapping][conn] +=1

In [ ]:
motif_fractions_actual = {}
for stage, net in network_dict.items():
    motif_fractions_actual[stage] = {}
    for mapping in mappings:
        motif_fractions_actual[stage][mapping] = np.array(num_graphs[stage][mapping])/(np.array(num_graphs[stage][mappings[0]]) + np.array(num_graphs[stage][mappings[1]]))

In [ ]:
cols = ['gray', 'orange']
f, ax = plt.subplots(figsize=(max_chain_length*2,2), ncols=max_chain_length, sharey=True, sharex=True)
for j, mapping in enumerate(mappings):
    by_length = {}
    for stage in motif_fractions_actual:
        for l in range(max_chain_length):
            if not l in by_length:
                by_length[l] = []
            by_length[l].append(motif_fractions_actual[stage][mapping][l])
    for l in range(max_chain_length):
        ax[l].plot(range(len(motif_fractions_actual.keys())), by_length[l], label=f'{mapping}', color=cols[j])
        ax[l].scatter(range(len(motif_fractions_actual.keys())), by_length[l], color=cols[j])
        ax[l].set_xticks(range(len(motif_fractions_actual.keys())), motif_fractions_actual.keys())
        utils.simpleaxis(ax[l])
        ax[l].set_title(l+1)
        ax[l].set_ylim((0,1))
plt.legend(loc='center right', bbox_to_anchor=(1.3,0.5), frameon=False)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(1.5,1.5))
utils.nx.draw(motif, with_labels=True, ax=ax, node_color='gray', node_size=225)
plt.show()

In [ ]:
piesize=0.08
chain_length = 3
for stage in network_dict.keys():
    for mapping in mappings:
        print(stage, mapping)
        hm = utils.make_hypermotifs(motif, chain_length, [mapping])
        pos = utils.nx.kamada_kawai_layout(hm)
        color_dict = {p:facecolors for p in pos.keys()}
        alpha_dict = {p:1 for p in pos.keys()}
        if sum(ntype_motif[stage][mapping][m].values()) == 0:
            continue
        pie_division = {m: [ntype_motif[stage][mapping][m][n]/sum(ntype_motif[stage][mapping][m].values()) for n in ntype] for m in hm.nodes}

        f, ax = plt.subplots(figsize=(8,8))
        utils.nx.draw_networkx_edges(hm, pos=pos, node_size=1200, connectionstyle='arc3', arrowstyle='->')
        for n,p in pos.items():
            utils.plot_pie(n=n, center=p, ax=ax, color_dict=color_dict, alpha_dict=alpha_dict, pie_division=pie_division[n], piesize=piesize)
        # utils.nx.draw_networkx_labels(hm, pos=pos)
        ax.set_xlim(-1, 1)
        ax.set_ylim(-1, 1)
        ax.set_aspect('equal')
        ax.axis('off')  # Optionally turn off axis
        # plt.savefig(f'motif_ntype_division_{mapping}.svg', transparent=True)
        plt.show()
        plt.close()


In [ ]:
for stage, net in network_dict.items():
    print(stage, len(net.neurons))